# QnA RAG
Q&A集を参照し、RAGで検索するプログラム。  
`RAG_QnA/docker/README.md`を参照し、Dockerコンテナ内で実行すること。  

データは日本の官公庁のWebサイトに掲載されている「よくある質問」を用いている。  
データ整備に尽力頂いたmatsuxr様に感謝する。  

In [1]:
# 必要なライブラリをインストール
import os
import pandas as pd
from dotenv import load_dotenv
import openai
import tiktoken
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from chromadb.config import Settings

# initial settings
load_dotenv()
API_KEYS = os.environ['API_KEYS']
GPT_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL = 'text-embedding-ada-002'
BATCH_SIZE = 1000

openai.api_key = API_KEYS

## APIキーの動作確認
APIキーの動作確認を行う  
方法は様々だが、ここでは'hello world'の`embedding`を生成するという方法で確認してみる。  
ベクトルが表示されればOK。  

In [ ]:
def create_embeddings_text(texts):
    embeddings = []
    for i in range(0, len(texts), BATCH_SIZE):
        batch = texts[i:i+BATCH_SIZE]
        response = openai.Embedding.create(
            model=EMBEDDING_MODEL,
            input=batch
        )
        embeddings.extend(response['data'])
    return embeddings

emb = create_embeddings_text('hello world')
print(emb)

## `token`を確認
入力できる`token`数には制限があるため、事前に確認する関数を用意  
ここで用いている`text-embedding-ada-002`の最大`token`数は`4097`  
データは簡単のため気象庁が発行したものに絞っている。  
幸い、気象庁が発行するQ&Aでは、`token`が4097をこえるものは無かった。

In [ ]:
# tokenを測定する関数を用意
def num_tokens(text, model=GPT_MODEL):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# data.jsonlをDataFrameで読み込み
path_data = os.path.join('data', 'data.jsonl')
df_data = pd.read_json(path_data, orient='records', lines=True)

# 'copyright'が気象庁のものだけピックアップ
df_data = df_data[df_data['copyright']=='気象庁']

# QuestionとAnswerを結合
df_data['QnA'] = df_data['Question'] + ' ' + df_data['Answer']
# token取得
df_data['token_QnA'] = df_data['QnA'].apply(num_tokens)

# tokenが4097以上のものは除外
df_data = df_data[df_data['token_QnA'] < 4097].reset_index(drop=True)

## `Embeddings`の生成
ベクトル検索をかけるための`Embeddings`を生成する。  

In [ ]:
def create_embeddings(items):
    embeddings = []
    for batch_start in range(0, len(items), BATCH_SIZE):
        batch_end = batch_start + BATCH_SIZE
        batch = items[batch_start:batch_end]
        print(f'Batch {batch_start} to {batch_end-1}')
        response = openai.Embedding.create(
            model=EMBEDDING_MODEL,
            input=batch
        )
        for i, be in enumerate(response['data']):
            assert i == be['index'] # double check embeddings are in same order as input
        batch_embeddings = [e['embedding'] for e in response['data']]
        embeddings.extend(batch_embeddings)

    df = pd.DataFrame({'text': items, 'embedding': embeddings})
    return df

# Embeddings
items = df_data['QnA'].to_list()
df_embedding = create_embeddings(items=items)

## データベース設定
データベースを設定する。  
ここでは`chromadb`を採用する。  

In [ ]:
def create_chroma_client():
    persist_directory = 'chroma_persistence'
    chroma_client = chromadb.Client(
        Settings(
            persist_directory=persist_directory,
            chroma_db_impl="duckdb+parquet",
        )
    )
    return chroma_client

def chroma_collection(chroma_client):
    chroma_client.reset()
    collection_name = 'stevie_collection'
    embedding_function = OpenAIEmbeddingFunction(api_key=API_KEYS, model_name=EMBEDDING_MODEL)
    collection = chroma_client.create_collection(name=collection_name, embedding_function=embedding_function)
    return collection

# settings for chromacb
chroma_client = create_chroma_client()
stevei_collection = chroma_collection(chroma_client)
stevei_collection.add(
    ids = df_embedding.index.astype(str).tolist(),
    documents=df_embedding['text'].tolist(),
    embeddings=df_embedding['embedding'].tolist(),
)
chroma_client.persist()

## データベースを検索する
ベクトル検索の設定。  
応答に加え、関連度も出力する。  

In [ ]:
def query_collection(
        query: str,
        collection: chromadb.api.models.Collection.Collection,
        max_results: int=100) -> tuple[list[str], list[float]]:
    results = collection.query(query_texts=query, n_results=max_results, include=['documents', 'distances'])
    strings = results['documents'][0]
    relatednesses = [1 - x for x in results['distances'][0]]
    return strings, relatednesses

# response
strings, relatednesses = query_collection(
    collection=stevei_collection,
    query='昨今の異常気象の原因を教えて下さい',
    max_results=3,
)

for string, relatednesses in zip(strings, relatednesses):
    print(f'{relatednesses=:.3f}')
    print(string)